In [ ]:
import open3d as o3d
import numpy as np
import copy
from os import listdir
import os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

from winsound import Beep

In [ ]:
def draw_point_clouds(point_clouds, black_point_cloud = None):
    pc_copy = []
    colors = [[1, 0.706, 0], [0, 0.651, 0.929], [0.3, 0.351, 0.529]]
    for idx, x in enumerate(point_clouds):
        temp = copy.deepcopy(x)
        if idx < len(colors):
            temp.paint_uniform_color(colors[idx])
        else:
            temp.paint_uniform_color(np.random.rand(3))
        pc_copy.append(temp)
    
    if black_point_cloud is not None:
        temp = copy.deepcopy(black_point_cloud)
        temp.paint_uniform_color([0,0,0])
        pc_copy.append(temp)
   
    o3d.visualization.draw_geometries(pc_copy)

In [ ]:
source_folder = "C:\\Users\\amoff\\Documents\\Meine Textdokumente\\Masterarbeit\\Daten\\rotated_and_translated\\100000000\\transformed_frames\\"

target_path = "C:\\Users\\amoff\\Documents\\Meine Textdokumente\\Masterarbeit\\LiDAR_Punktwolke-RevA-201455_Vermessungsdaten\\Cleaned Export\\intersection_dirty\\intersection_dirty2_xyzi_down_transformed_compressed.pcd"
target = o3d.io.read_point_cloud(target_path)

In [ ]:
pcd_down = target.voxel_down_sample(voxel_size=1)

arr = [pcd_down]
arr.append(o3d.io.read_point_cloud(source_folder + "1667908110_100000000_rat_xyzi.pcd"))
arr.append(o3d.io.read_point_cloud(source_folder + "1667908113_100000000_rat_xyzi.pcd"))
arr.append(o3d.io.read_point_cloud(source_folder + "1667908118_100000000_rat_xyzi.pcd"))
arr.append(o3d.io.read_point_cloud(source_folder + "1667908122_100000000_rat_xyzi.pcd"))

draw_point_clouds(arr)

In [ ]:


# ax.margins(x=0, y=-0.25)

In [ ]:
column = np.asarray(arr[1].points)[:,2]
column

In [ ]:
df = pd.DataFrame(column)
df.hist(bins=50)


In [ ]:
frame = arr[1]
column = np.asarray(frame.points)[:,2]
filtered_frame = np.asarray(frame.points)[column > 0]

In [ ]:
df = pd.DataFrame(filtered_frame)
df.hist(bins=50)

In [ ]:
filtered_frame.shape

In [ ]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(filtered_frame)

In [ ]:
draw_point_clouds([pcd])

In [ ]:

def remove_ground(point_cloud, ground_height=0):
    column = np.asarray(point_cloud.points)[:,2]
    print("Number of points before filtering", column.shape)
    filtered_frame = np.asarray(point_cloud.points)[column > ground_height]
    print("Number of points after filtering", filtered_frame.shape)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(filtered_frame)
    return pcd

In [ ]:

def crop(point_cloud, dimension = 2, lower_bound = 0, upper_bound = None):
    column = np.asarray(point_cloud.points)[:,dimension]
    filtered_frame = np.asarray(point_cloud.points)[column > lower_bound ]
    if upper_bound is not None:
        column = filtered_frame[:,dimension]
        filtered_frame = filtered_frame[column < upper_bound]

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(filtered_frame)
    return pcd

In [ ]:
draw_point_clouds([remove_ground(arr[0]),remove_ground(arr[1]),remove_ground(arr[2])])

Remove ground from all frames

In [ ]:
src_folder = "C:\\Users\\amoff\\Documents\\Meine Textdokumente\\Masterarbeit\\Daten\\rotated_and_translated\\100000000\\transformed_frames\\"
target_folder = "C:\\Users\\amoff\\Documents\\Meine Textdokumente\\Masterarbeit\\Daten\\rotated_and_translated\\100000000\\ground_removed_frames\\"
target_suffix = "_ground_removed.pcd"
filenames = listdir(src_folder)

for idx, file in enumerate(filenames):
    print(str(idx) + " / " + str(len(filenames)) + " " + src_folder+file)
    point_cloud = o3d.io.read_point_cloud(src_folder+file)
    f = file.split("_")
    base_name = f[0] + "_" + f[1]
    
    point_cloud = remove_ground(point_cloud, 1)
    o3d.io.write_point_cloud(target_folder + base_name + target_suffix, point_cloud, write_ascii = True )

In [ ]:
src_file = "C:\\Users\\amoff\\Documents\\Meine Textdokumente\\Masterarbeit\\LiDAR_Punktwolke-RevA-201455_Vermessungsdaten\\Cleaned Export\\intersection_dirty\\intersection_dirty2_xyzi_down_transformed_compressed.pcd"
target_file = "C:\\Users\\amoff\\Documents\\Meine Textdokumente\\Masterarbeit\\LiDAR_Punktwolke-RevA-201455_Vermessungsdaten\\Cleaned Export\\intersection_dirty\\intersection_dirty2_xyzi_down_transformed_compressed_ground_removed.pcd"
point_cloud = o3d.io.read_point_cloud(src_file)

point_cloud = remove_ground(point_cloud)
o3d.io.write_point_cloud(target_file, point_cloud, write_ascii = True )

CROP

In [ ]:
voxel_size = 1
target1 = target.voxel_down_sample(voxel_size)
draw_point_clouds([target1])

In [ ]:
target2 = remove_ground(target1, 1)
draw_point_clouds([target2])

In [ ]:
lx, ux, ly, uy, ground_height = -80, -10, -30, 30, 3

# target3 = crop(target1, 0, lx, ux)
# target3 = crop(target3, 1, ly, uy)
pcd = crop(target1, 2, ground_height)

draw_point_clouds([pcd])

In [ ]:
def crop3d(point_cloud):
    lx, ux, ly, uy, ground_height = -80, -10, -30, 30, 2
    pcd = crop(point_cloud, 0, lx, ux)
    pcd = crop(pcd, 1, ly, uy)
    return crop(pcd, 2, ground_height)
    

In [ ]:
src_folder = "C:\\Users\\amoff\\Documents\\Meine Textdokumente\\Masterarbeit\\Daten\\rotated_and_translated\\100000000\\transformed_frames\\"
target_folder = "C:\\Users\\amoff\\Documents\\Meine Textdokumente\\Masterarbeit\\Daten\\rotated_and_translated\\100000000\\cropped_frames\\"
target_suffix = "_croped.pcd"
filenames = listdir(src_folder)

for idx, file in enumerate(filenames):
    print(str(idx) + " / " + str(len(filenames)) + " " + src_folder+file)
    point_cloud = o3d.io.read_point_cloud(src_folder+file)
    f = file.split("_")
    base_name = f[0] + "_" + f[1]
    
    point_cloud = crop3d(point_cloud)
    o3d.io.write_point_cloud(target_folder + base_name + target_suffix, point_cloud, write_ascii = True )

In [ ]:
target_path = "C:\\Users\\amoff\\Documents\\Meine Textdokumente\\Masterarbeit\\LiDAR_Punktwolke-RevA-201455_Vermessungsdaten\\Cleaned Export\\intersection_dirty\\intersection_dirty2_xyzi_down_transformed_compressed.pcd"
target = o3d.io.read_point_cloud(target_path)
point_cloud = crop3d(target)
target_path = "C:\\Users\\amoff\\Documents\\Meine Textdokumente\\Masterarbeit\\LiDAR_Punktwolke-RevA-201455_Vermessungsdaten\\Cleaned Export\\intersection_dirty\\intersection_dirty2_xyzi_down_transformed_compressed_cropped.pcd"
o3d.io.write_point_cloud(target_path, point_cloud, write_ascii = True )